## ESP_2025 - APLICAÇÃO 1 - TRIBUTAÇÃO ÓTIMA DO CONSUMO

In [26]:
# bibliotecas usadas
import numpy as np
import pandas as pd

In [27]:
caminho_arquivo = r"C:\{estudos}\ESP\Economia-do-Setor-Publico-2025.2\Aplicação 1\ESP_2025_aplicação 1_POF 2017-2018.xlsx"
df = pd.read_excel(caminho_arquivo, sheet_name=0) 

In [28]:
# Verificar as colunas disponíveis
print("Colunas disponíveis no arquivo:")
print(df.columns.tolist())
print(f"\nNúmero de observações: {len(df)}")

# Verificar se há valores missing
print("\nValores missing por coluna:")
print(df.isnull().sum())

Colunas disponíveis no arquivo:
['ID_DOM', 'PESO', 'NUM_MOR', 'CB', 'OA', 'FUMO', 'BA', 'VE', 'EE_CD', 'AL', 'BS', 'SA', 'TPA', 'TPU', 'COM', 'ED', 'RC', 'HCP', 'OBS']

Número de observações: 57920

Valores missing por coluna:
ID_DOM     0
PESO       0
NUM_MOR    0
CB         0
OA         0
FUMO       0
BA         0
VE         0
EE_CD      0
AL         0
BS         0
SA         0
TPA        0
TPU        0
COM        0
ED         0
RC         0
HCP        0
OBS        0
dtype: int64


### 2) CÁLCULO DO GASTO DOMICILIAR TOTAL

In [29]:
colunas_gasto = ['CB', 'OA', 'FUMO', 'BA', 'VE', 'EE_CD', 'AL', 'BS', 'SA', 
                 'TPA', 'TPU', 'COM', 'ED', 'RC', 'HCP', 'OBS']

# Verificar quais colunas estão presentes
colunas_presentes = [col for col in colunas_gasto if col in df.columns]
print(f"\nColunas de gasto presentes: {colunas_presentes}")

df['GASTO'] = df[colunas_presentes].sum(axis=1)
print(f"Gasto total calculado para {len(df)} domicílios")



Colunas de gasto presentes: ['CB', 'OA', 'FUMO', 'BA', 'VE', 'EE_CD', 'AL', 'BS', 'SA', 'TPA', 'TPU', 'COM', 'ED', 'RC', 'HCP', 'OBS']
Gasto total calculado para 57920 domicílios


### 3) CÁLCULO DO GASTO DOMICILIAR TOTAL PER CAPITA


In [30]:
df['GASTO_PC'] = df['GASTO'] / df['NUM_MOR']
df['GASTO_PC']

0         1263.447778
1         1214.552083
2          662.919584
3         2374.151000
4          746.548667
             ...     
57915    18590.670002
57916     4322.568125
57917     5484.149722
57918     1522.797500
57919     3053.888333
Name: GASTO_PC, Length: 57920, dtype: float64

### 4) CÁLCULO DAS PARCELAS ORÇAMENTÁRIAS MÉDIAS

In [31]:
categorias = {
    'CB': 'CESTA BÁSICA',
    'OA': 'OUTROS ALIMENTOS', 
    'FUMO': 'FUMO',
    'BA': 'BEBIDAS ALCOÓLICAS',
    'VE': 'VESTUÁRIO',
    'EE_CD': 'ENERGIA ELÉTRICA E COMBUSTÍVEIS DOMÉSTICOS',
    'AL': 'ALUGUEL',
    'BS': 'BENS E SERVIÇOS DOMÉSTICOS',
    'SA': 'SAÚDE',
    'TPA': 'TRANSPORTE PARTICULAR',
    'TPU': 'TRANSPORTE PÚBLICO',
    'COM': 'COMUNICAÇÃO',
    'ED': 'EDUCAÇÃO',
    'RC': 'RECREAÇÃO E CULTURA',
    'HCP': 'HIGIENE E CUIDADOS PESSOAIS',
    'OBS': 'OUTROS BENS E SERVIÇOS'
}

# Calcular parcelas orçamentárias para todas as colunas presentes
for coluna in colunas_presentes:
    df[f'PO_ME_{coluna}'] = df[coluna] / df['GASTO']

print(f"Parcelas orçamentárias calculadas para {len(colunas_presentes)} categorias")


Parcelas orçamentárias calculadas para 16 categorias


### 5) CÁLCULO DO VALOR MÍNIMO DE GASTO DOMICILIAR TOTAL

In [32]:
gasto_minimo = df['GASTO'].min()
gasto_pc_minimo = df['GASTO_PC'].min()

print(f"\n--- VALORES MÍNIMOS DE GASTO ---")
print(f"Gasto domiciliar total mínimo: R$ {gasto_minimo:.2f}")
print(f"Gasto domiciliar per capita mínimo: R$ {gasto_pc_minimo:.2f}")


--- VALORES MÍNIMOS DE GASTO ---
Gasto domiciliar total mínimo: R$ 29.02
Gasto domiciliar per capita mínimo: R$ 10.29


### 6) CÁLCULO DO VALOR DO GASTO TOTAL DE SUBSISTÊNCIA

In [33]:
# Opção 1: 90% do menor gasto total
gasto_sub_1 = gasto_minimo * 0.9
df['GASTO_SUB'] = gasto_sub_1

# Opção 2: 90% do menor gasto per capita  
gasto_sub_2 = gasto_pc_minimo * 0.9
df['GASTO_SUB_2'] = gasto_sub_2

# Opção 3: Linha de pobreza (1/4 do salário mínimo de 2018)
salario_minimo_2018 = 954.00
gasto_sub_3 = salario_minimo_2018 / 4
df['GASTO_SUB_3'] = gasto_sub_3

print(f"\n--- GASTOS DE SUBSISTÊNCIA ---")
print(f"🏠 Opção 1 (90% do menor gasto total): R$ {gasto_sub_1:.2f}")
print(f"👤 Opção 2 (90% do menor gasto per capita): R$ {gasto_sub_2:.2f}")
print(f"📊 Opção 3 (linha de pobreza - 1/4 SM): R$ {gasto_sub_3:.2f}")


--- GASTOS DE SUBSISTÊNCIA ---
🏠 Opção 1 (90% do menor gasto total): R$ 26.12
👤 Opção 2 (90% do menor gasto per capita): R$ 9.26
📊 Opção 3 (linha de pobreza - 1/4 SM): R$ 238.50


### 7) CRIAÇÃO DOS QUINTIS DE GASTO TOTAL DOMICILIAR PER CAPITA

In [34]:
quintis = df['GASTO_PC'].quantile([0.2, 0.4, 0.6, 0.8])
print(f"\n--- PONTOS DE CORTE DOS QUINTIS ---")
for i, (q, valor) in enumerate(quintis.items(), 1):
    print(f"Quintil {i}: R$ {valor:.2f}")


--- PONTOS DE CORTE DOS QUINTIS ---
Quintil 1: R$ 431.68
Quintil 2: R$ 701.93
Quintil 3: R$ 1071.81
Quintil 4: R$ 1765.52


### 8) CRIAÇÃO DA VARIÁVEL QUINTIL DE GASTO TOTAL DOMICILIAR PER CAPITA

In [35]:
df['QUINTIL'] = pd.cut(df['GASTO_PC'], 
                      bins=[0, quintis[0.2], quintis[0.4], quintis[0.6], quintis[0.8], np.inf],
                      labels=[1, 2, 3, 4, 5])

# Distribuição por quintil
distribuicao_quintil = df['QUINTIL'].value_counts().sort_index()
print(f"\n--- DISTRIBUIÇÃO POR QUINTIL ---")
print(distribuicao_quintil)


--- DISTRIBUIÇÃO POR QUINTIL ---
QUINTIL
1    11584
2    11584
3    11584
4    11584
5    11584
Name: count, dtype: int64


## RELATÓRIO

Um relatório deve ser enviado, até a data limite estabelecida, contendo os seguintes resultados:

 (i) Estatísticas descritivas (média, desvio padrão, valor mínimo e valor máximo) referentes ao número de moradores dos domicílios, a cada grupo de consumo, ao gasto total e ao gasto per capita.

 (ii) Análise de como os procedimentos realizados acimas e os produtos gerados podem ajudar a discussão sobre a elaboração de um desenho tributário para o Brasil.



#### ` (i) Estatísticas descritivas completas`

In [39]:
variaveis_descritivas = ['NUM_MOR'] + colunas_presentes + ['GASTO', 'GASTO_PC']

print("\n" + "="*80)
print("📊 ESTATÍSTICAS DESCRITIVAS COMPLETAS")
print("="*80)

descricoes = df[variaveis_descritivas].describe()

for coluna in descricoes.columns:
    descricoes[coluna] = descricoes[coluna].apply(lambda x: f'{x:.2f}' if isinstance(x, (int, float)) else x)
print(descricoes)



📊 ESTATÍSTICAS DESCRITIVAS COMPLETAS
        NUM_MOR        CB        OA      FUMO        BA        VE     EE_CD  \
count  57920.00  57920.00  57920.00  57920.00  57920.00  57920.00  57920.00   
mean       3.08    264.53    317.59     14.41     10.79    139.35    163.50   
std        1.55    313.96    492.51     53.99     58.04    225.58    119.93   
min        1.00      0.00      0.00      0.00      0.00      0.00      0.00   
25%        2.00     38.52     34.49      0.00      0.00     13.97     89.56   
50%        3.00    168.44    152.28      0.00      0.00     70.06    137.09   
75%        4.00    375.83    407.31      0.00      0.00    179.30    203.14   
max       17.00   4704.70  18281.30   1776.67   2134.82  12809.09   2063.40   

             AL        BS        SA       TPA       TPU       COM        ED  \
count  57920.00  57920.00  57920.00  57920.00  57920.00  57920.00  57920.00   
mean     580.92    285.16    256.84    496.37    109.88    117.44    117.38   
std      637.

In [41]:
# Estatísticas adicionais por quintil
print("\n" + "="*80)
print("💰 GASTO MÉDIO POR QUINTIL")
print("="*80)
gasto_por_quintil = df.groupby('QUINTIL')['GASTO_PC'].agg(['mean', 'std', 'min', 'max', 'count'])
print(gasto_por_quintil)

# Parcelas orçamentárias médias por quintil
print("\n" + "="*80)
print("📈 PARCELAS ORÇAMENTÁRIAS MÉDIAS POR QUINTIL")
print("="*80)

# Calcular médias das parcelas orçamentárias por quintil
colunas_po = [f'PO_ME_{col}' for col in colunas_presentes]
parcelas_por_quintil = df.groupby('QUINTIL')[colunas_po].mean()

# Formatar para melhor visualização
for col in colunas_presentes:
    po_col = f'PO_ME_{col}'
    if po_col in parcelas_por_quintil.columns:
        print(f"\n📦 {categorias.get(col, col)}:")
        for quintil in range(1, 6):
            if quintil in parcelas_por_quintil.index:
                valor = parcelas_por_quintil.loc[quintil, po_col]
                print(f"  Quintil {quintil}: {valor:.2f} ({valor*100:.2f}%)")


💰 GASTO MÉDIO POR QUINTIL
                mean          std          min           max  count
QUINTIL                                                            
1         288.227787    93.495856    10.289167    431.650000  11584
2         563.170248    78.047522   431.686042    701.912500  11584
3         872.589862   105.267695   701.940417   1071.755001  11584
4        1368.326530   196.013680  1071.887500   1765.500000  11584
5        3489.990123  2536.340335  1765.615416  68757.390833  11584

📈 PARCELAS ORÇAMENTÁRIAS MÉDIAS POR QUINTIL

📦 CESTA BÁSICA:
  Quintil 1: 0.13 (12.77%)
  Quintil 2: 0.11 (11.47%)
  Quintil 3: 0.10 (10.48%)
  Quintil 4: 0.09 (9.17%)
  Quintil 5: 0.06 (6.16%)

📦 OUTROS ALIMENTOS:
  Quintil 1: 0.07 (6.94%)
  Quintil 2: 0.09 (8.52%)
  Quintil 3: 0.10 (9.53%)
  Quintil 4: 0.10 (10.24%)
  Quintil 5: 0.10 (9.71%)

📦 FUMO:
  Quintil 1: 0.01 (0.74%)
  Quintil 2: 0.01 (0.65%)
  Quintil 3: 0.01 (0.63%)
  Quintil 4: 0.01 (0.55%)
  Quintil 5: 0.00 (0.37%)

📦 BEBIDAS 

C:\Users\Madu\AppData\Local\Temp\ipykernel_44668\3097870317.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  gasto_por_quintil = df.groupby('QUINTIL')['GASTO_PC'].agg(['mean', 'std', 'min', 'max', 'count'])
C:\Users\Madu\AppData\Local\Temp\ipykernel_44668\3097870317.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  parcelas_por_quintil = df.groupby('QUINTIL')[colunas_po].mean()


In [42]:

# Análise das parcelas orçamentárias médias gerais
print("\n" + "="*80)
print("📊 PARCELAS ORÇAMENTÁRIAS MÉDIAS GERAIS")
print("="*80)
for col in colunas_presentes:
    po_col = f'PO_ME_{col}'
    media_geral = df[po_col].mean()
    print(f"{categorias.get(col, col):<45}: {media_geral:.4f} ({media_geral*100:.2f}%)")


📊 PARCELAS ORÇAMENTÁRIAS MÉDIAS GERAIS
CESTA BÁSICA                                 : 0.1001 (10.01%)
OUTROS ALIMENTOS                             : 0.0899 (8.99%)
FUMO                                         : 0.0059 (0.59%)
BEBIDAS ALCOÓLICAS                           : 0.0026 (0.26%)
VESTUÁRIO                                    : 0.0423 (4.23%)
ENERGIA ELÉTRICA E COMBUSTÍVEIS DOMÉSTICOS   : 0.0714 (7.14%)
ALUGUEL                                      : 0.2142 (21.42%)
BENS E SERVIÇOS DOMÉSTICOS                   : 0.0808 (8.08%)
SAÚDE                                        : 0.0722 (7.22%)
TRANSPORTE PARTICULAR                        : 0.0974 (9.74%)
TRANSPORTE PÚBLICO                           : 0.0333 (3.33%)
COMUNICAÇÃO                                  : 0.0333 (3.33%)
EDUCAÇÃO                                     : 0.0233 (2.33%)
RECREAÇÃO E CULTURA                          : 0.0383 (3.83%)
HIGIENE E CUIDADOS PESSOAIS                  : 0.0592 (5.92%)
OUTROS BENS E SERVIÇOS      

#### ` (ii) Análise para tributação ótima do consumo`
- Exportando os dados para fazer a análise

In [38]:
caminho_saida = "resultados_analise_pof_completo.xlsx"
with pd.ExcelWriter(caminho_saida) as writer:
    df.to_excel(writer, sheet_name='Dados_Completos', index=False)
    descricoes.to_excel(writer, sheet_name='Estatisticas_Descritivas')
    gasto_por_quintil.to_excel(writer, sheet_name='Gasto_Por_Quintil')
    parcelas_por_quintil.to_excel(writer, sheet_name='Parcelas_Orcamentarias')
    
    # Adicionar resumo executivo
    resumo = pd.DataFrame({
        'Metrica': [
            'Total de domicílios',
            'Gasto total médio',
            'Gasto per capita médio', 
            'Número médio de moradores',
            'Gasto mínimo total',
            'Gasto mínimo per capita',
            'Gasto de subsistência (Opção 1)',
            'Gasto de subsistência (Opção 2)',
            'Gasto de subsistência (Opção 3)'
        ],
        'Valor': [
            len(df),
            f"R$ {df['GASTO'].mean():.2f}",
            f"R$ {df['GASTO_PC'].mean():.2f}",
            f"{df['NUM_MOR'].mean():.2f}",
            f"R$ {gasto_minimo:.2f}",
            f"R$ {gasto_pc_minimo:.2f}",
            f"R$ {gasto_sub_1:.2f}",
            f"R$ {gasto_sub_2:.2f}",
            f"R$ {gasto_sub_3:.2f}"
        ]
    })
    resumo.to_excel(writer, sheet_name='Resumo_Executivo', index=False)

print(f"\nResultados exportados para: {caminho_saida}")

# Resumo executivo final
print("\n" + "="*80)
print("RESUMO EXECUTIVO")
print("="*80)
print(f"Total de domicílios analisados: {len(df):,}")
print(f"Gasto total médio: R$ {df['GASTO'].mean():.2f}")
print(f"Gasto per capita médio: R$ {df['GASTO_PC'].mean():.2f}")
print(f"Número médio de moradores: {df['NUM_MOR'].mean():.2f}")
print(f"Faixa de gasto per capita: R$ {df['GASTO_PC'].min():.2f} a R$ {df['GASTO_PC'].max():.2f}")
print(f"Número de categorias de consumo analisadas: {len(colunas_presentes)}")


Resultados exportados para: resultados_analise_pof_completo.xlsx

RESUMO EXECUTIVO
Total de domicílios analisados: 57,920
Gasto total médio: R$ 3373.48
Gasto per capita médio: R$ 1316.46
Número médio de moradores: 3.08
Faixa de gasto per capita: R$ 10.29 a R$ 68757.39
Número de categorias de consumo analisadas: 16
